# Final project

## Project title

Research Question: Which UvA courses do I meet the entry requirements for? XXX






## Process Book

1. Don’t forget the process book. Maintain it every day you’ve worked on the project. This is the main way for 
   us to see what you’ve learned and is an important part of the evaluation.
2. Don’t use it for everything. Don’t spend too much time on your process book! Ten minutes 
   every day you’ve worked on the project on this, should be more than enough.
3. Keep it simple and short. Use simple language, short sentences, and be as straightforward as possible when 
   documenting your learning process.
4. Create a simple template. In order to organize your process book, it is nice to create and use a template:


### Date of entry: 25. Jan
#### What I’ve worked on: 
Git
- Lecture git
- Tried out git by myself
- Created git repository

Project choice
- Learn to use tableau and uber data as I am applying to intern at uber and tableau is a desired skill

Tableau
- Downloaded tableau 
- See what I can use tableau for to decide which project I want to do specifically
- Learn tableau basics
- Experiment with DP datasets (e.g., weather data) in tableau
- Experiment with uber selection dataset --> too easy (?)
    
#### What problems I encountered:
- Cannot use dataset I first thought about using

#### What I learned:
- How to use git
- Tableau basics https://www.youtube.com/watch?v=jEgVto5QME8
- I want to gather my own uber data e.g., through web-scraping, or API

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide, uber data XXX




### Date of entry: 26. Jan
#### What I’ve worked on: 
Tableau
- Learned more functions of tableau and experimented with these  

Project
- Created jupyter notebook
- Started process book

Brainstorm
- Potentially make website with tableau dashboard
- Which data can I use? 

Find data
- Look for uber APIs

#### What problems I encountered: 
- Uber APIs are not open, I can not use them 

#### What I learned: 
Tableau
- Basic functions: https://www.youtube.com/watch?v=yejulE4b-3Y
- Use and join (csv) files: https://www.youtube.com/watch?v=5WXrJ1JvTOo
- Integrate tableau dashboard in website: https://www.youtube.com/watch?v=wJ2CHIJalNU

#### Which resources did I use:
- Course material, YouTube, tableau, tableau user guide



### Date of entry: 27. Jan
#### What I’ve worked on: 
- Find new project ideas
- Decided for final topic/Question
- Changed strategy for scraping data: work with individual course pages instead of course overview

#### What problems I encountered: 
- Url of studiegids does not change per page of courses

#### What I learned: 

#### Which resources did I use:
- Multiple forms to try to solve the url problem, but didn't find a solution




### Date of entry: 30. Jan
#### What I’ve worked on: 
- Scraping course data: name of course, url, fiel of studies, education level, entry requirements
- Started transforming the entry requirements text into categorical entry requirement checks that students can filter for

#### What problems I encountered: 
- I cannot simply check whether a unique number-combination in the end of the link opens a specific course page by testing if the link is dead or not: if the number in the end doesn't link to a course it still opens the studigids page
- The entry requirement texts are very heterogenous and need a lot of transformation. This will become the most time-consuming part of scraping


#### What I learned: 
- New commands for text transformation

#### Which resources did I use:





### Date of entry: 31. Jan
#### What I’ve worked on: 
Refined scraping entry requirements (scraper 2):
- Added function (1) to prepare/transform entry requirements text for further functions
- Added function (2) to extract courses and with specific credit requirements
- Added function (3) to extract all courses required 
- Combined these functions in one entry requirements-scraper

#### What problems I encountered: 
- Function (3) often required adjustments as it recognised some 'course names' incorrectly
- I could integrate a feedback function

#### What I learned: 

#### Which resources did I use:
- nltk to detect stopwords




### Date of entry: 1. Feb
#### What I’ve worked on: 
Refined scraping entry requirements (scraper 2):
- Improved function (2) to extract courses and with specific credit requirements
- Experimented with combining scraper part 1 with scraper part 2 (entry requirements)

#### What problems I encountered: 
- Transform entry requirements text to categories: Special cases for scraping course that has credit requirement after the course name ('course name (XX EC)') instead of before ('XX ec in course name)

#### What I learned: 
- New commands for text processing
- New ways of slicing/working with lists

#### Which resources did I use:



### Date of entry: 2. Feb
#### What I’ve worked on: 
- Making final combined scraper
- Fixing entry requirements errors

#### What problems I encountered: 

#### What I learned: 

#### Which resources did I use:


forums
stackoverflow

## To-Do

1. Scrape / Transform data 
2. Import data into tableau
3. (Webpage with tableau dashboard)

## 1. Data

Relevant personal data

Programme: ""
Level: Bachelor's, Master's (Master's, Master), Exchange programme ('Exchange Programme Exchange Programme UvA', 'Exchange Programme Exchange Programme'), pre-Master's programme, Secondary Subject/Bijvak('?'), 'Honours MSc', 'Research Master's
Year of the studies: ""
Credits obtained this/1st year: ""

url: https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course


Relevant course information 

Course name: Course name
Entry requirements: Course > Entry requirements > Split by sentence & look for keywords (if not present, add to dict with unknown entry requirements)
Level: Course > Is part of > loop through Level list (s.a.) 
Field: Course > Is part of > remove Level > everything left 

(Coordinator name: Course > Lecturer(s) > Coordinator ('co-ordinator')
(Coordinator email: Course > Lecturer(s) > Coordinator (crawl) > Email)


In [95]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing

from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import numpy as np
from math import ceil
import sys
import nltk
nltk.download('punkt')
nltk.download('stopwords')



ModuleNotFoundError: No module named 'validators'

In [2]:
#Prepare webscraping with BeautifulSoup

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
        }
        with closing(get(url, stream=True, headers=headers)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        print('The following error occurred during HTTP GET request to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and content_type is not None
            and content_type.find('html') > -1)

- All courses have individual descriptions with unique urls
- First, I will find all urls that link to courses by trying out urls
- I can check whether I found all courses by comparing the length of my url list to the number or courses stated on the website
- After that, I will loop through the urls that work and scrape the data for all courses

In [239]:
#SCRAPER PART 1: name, urls, levels and fields (subset of the data) 

potential_levels = ["Bachelor's", "Dual Master's", "Master's", "Exchange Programme Exchange Programme UvA", 
                    "Exchange programme Exchange Programme", "Exchange programme Exchange Programme UvA", 
                    "Exchange Programme Exchange Programme", "pre-Master's programme", "Secondary Subject", 
                    "Bijvak", "Honours MSc", "Research Master's", "Minor"] #tailored to english website

courses = {}
unique_n = 102590
    
while unique_n < 102600:
    url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/{unique_n}'
    html = simple_get(url)
    dom = BeautifulSoup(html, 'html.parser')
    
    unique_n += 1
    
    if (dom.find('article', class_ = 'twelve columns')) is None: #no course under this url - go to next url
        pass
    
    else: #course exists
        name = dom.find('article', class_ = 'twelve columns').h1.text[1::]
        courses[name] = {}
        #print(name)
        courses[name]['url'] = url
        courses[name]['Academic level'] = [] #Add empty list in case there is no data
        courses[name]['Field of study'] = [] #Add empty list in case there is no data
        courses[name]['Entry requirements'] = [] #Add empty list in case there is no data

        #print(url)
        
        for info_block in dom.find_all('tr', {'class':''}):
            info_block = info_block.text


            #Education level and field of the course
            if 'Is part of' in info_block: #Find text box that contains level and field of the course
                is_part_of = info_block.replace("Is part of", "") #Remove title of the text box

                # Education level of the course e.g., Bachelor's
                levels = [] 
                for l in potential_levels:
                    if l in is_part_of:
                        levels.append(l)

                # Field(s) of the course e.g., Business
                for lev in levels: 
                    is_part_of = is_part_of.replace(lev+" ", ";") #replace education level by semicolon to split on the semicolon later
                fields = is_part_of[1::].split(";") #remove first semicolon to not get an empty element in the study fields list 
                                                    #and split on semicolon to get this list
        
                courses[name]['Academic level'] = levels
                courses[name]['Field of study'] = fields

        
        #entry = 

print(courses)

{'Work, Power & Income:\xa0Why regulation fails to address labour market inequality (&sometimes succeeds)': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102591', 'Academic level': ["Bachelor's"], 'Field of study': ['Politics, Psychology, Law and Economics (PPLE)'], 'Entry requirements': []}, 'American Think Tanks and U.S. Foreign Policy ': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102593', 'Academic level': ['Exchange programme Exchange Programme'], 'Field of study': ['Graduate School of Humanities (MA)'], 'Entry requirements': []}, '"Losing Earth"?: Activism and Diplomacy on the Environment and Climate since 1968': {'url': 'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595', 'Academic level': ['Exchange programme Exchange Programme'], 'Field of study': ['Graduate School of Humanities (MA)'], 'Entry requirements': []}}


In [ ]:
#indicators = 'priority', 'research participation', 'only', 'selected into', 
#              'completed', 'major/minor', 'major', 'minor', 'elective' XXX



In [73]:
# Run before testing single functions

url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/101959'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')



In [74]:
#Test: clean_entry function

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text
    
    # Entry requirements of the course: full text
    if 'Entry requirements' in info_block: #Find text box with entry requirements
        entry = info_block.replace("Entry requirements", "") #Remove title of the text box
        entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining

        #Split into sentences as these contain seperate criteria
        entry = entry.replace(". ", ".").split(".")
        entry = list(filter(None, entry))

print(entry)   

['60 ec from the first-year Psychology curriculum, including Research Participation', 'Scientific and Statistical Reasoning (7202A701XY), Practical training: Psychological Research (7202A704), as well as ≥ 18 ec of the course package of either Brain & Cognition or Psychological Methods', 'For Brain & Cognition students, the 18 EC must include the course B&C Toolbox: Experimental and Neuroscientific Methods (7202BP04)']


In [80]:
# LOWERCASE FIRST/SECOND/THIRD YEAR AND ECS - finished

def clean_entry(entry):
    low = ['Year', 'EC', 'First', 'Second', 'Third', 'First-Year', 'First-year', 'first-Year', # List of words that need to be lowercase
           'Second-Year', 'Second-year', 'second-Year', 'Third-Year', 'Third-year', 'third-Year',
           'EC', 'ECTs', 'Credits', 'ECTS', 'ECT', 'Ect', 'Ec', 'ECs', 'Ecs'] 
    low_2 = ['EC)', 'ECTs)', 'Credits)', 'ECTS)', 'ECT)', 'Ect)', 'Ec)', 'ECs)', 'Ecs)'] 
    low_3 = ['EC),', 'ECTs),', 'Credits),', 'ECTS),', 'ECT),', 'Ect),', 'Ec),', 'ECs),', 'Ecs),']
    error_wordsl = ['training:'] #words I found that were incorrectly lowercased

    entry_c = '' #transformed entry
    #print(entry)

    
    for sentence in entry:
        words = sentence.split() #split entry into sentences and into words

        for word in words:
            if word in low:
                word_l = word.lower() #Transform first/second/third year to lowercase so it doesn't interrupt the course title detection
                entry_c += word_l+" "
            elif word in low_2:
                word_l = word.lower()
                word_l = word_l[:-1]
                entry_c += word_l+" "
            elif word in low_3:
                word_l = word.lower()
                word_l = word_l[:-2] #XXX offene Klammer; dann nach links anstatt rechts
                entry_c += word_l+" "
            elif word in error_wordsl:
                word_l = word[0].upper() + word[1:]
                entry_c += word_l+" "
            else:
                entry_c += word+ " "
        entry_c = entry_c[:-1]+"." #remove last space

    entry_c = entry_c.split('.') #turn entry form full-text into list of sentences again
    entry_c = list(filter(None, entry_c)) #remove empty elements

    return entry_c

entry = clean_entry(entry)
print(entry)


['60 ec from the first-year Psychology curriculum, including Research Participation', 'Scientific and Statistical Reasoning (7202A701XY), Practical Training: Psychological Research (7202A704), as well as ≥ 18 ec of the course package of either Brain & Cognition or Psychological Methods', 'For Brain & Cognition students, the 18 ec must include the course B&C Toolbox: Experimental and Neuroscientific Methods (7202BP04)']


In [8]:
#Test: find_c_names function

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
#sentence = 'Scientific and Statistical Reasoning (7202A701XY), Practical Training: Psychological Research (7202A704), ≥ 18 ec of the course package Clinical Psychology and the first part of Evidence-based Clinical Practice: Research Methods/CP (72037001).'
sentence = '60 ec from the first-year Psychology curriculum, including Research Participation'
sentence = 'Scientific and Statistical Reasoning (7202A701XY), Practical Training: Psychological Research (7202A704), ≥ 18 ec of the course package Clinical Psychology and the first part of Evidence-based Clinical Practice: Research Methods/CP (72037001)'
sentence = 'first academic year in Anthropology (60 ec Theory and History of Anthropology (THA) and/or Historical and Comparative Sociology (HCS), a Specialisation Course and Practicing Ethnography'
words = sentence.split()
print(words)

['first', 'academic', 'year', 'in', 'Anthropology', '(60', 'ec', 'Theory', 'and', 'History', 'of', 'Anthropology', '(THA)', 'and/or', 'Historical', 'and', 'Comparative', 'Sociology', '(HCS),', 'a', 'Specialisation', 'Course', 'and', 'Practicing', 'Ethnography']


In [70]:
#FUNCTION: COURSE TITLES - finished



# Extract list of course titles from the entry requirements text; based on capitalisation; sensitive to words
# that are typically not capitalised in titles (conjunctions, articles, prepositions)


def find_c_names(words): #input: list of words of one sentence from the entry requirements text
    
    # List of words not capitalised in headings
    conjunctions = ["and", "or", "but", "nor", "yet", "so", "for", "&"]
    articles = ['a', 'an', 'the']
    prepositions = ["in", "to", "of", "at", "by", "up", "for", "off", "on"]
    years = ['year', 'first-year', 'second-year', 'third-year'] 
    years_cond = ['first', 'second', 'third', '1st', '2nd', '3rd', 'academic', 'Academic'] #needs to be separate because of problem: Clinical Psychology and the first
    no_cap = conjunctions + articles + prepositions
    stop_words = set(nltk.corpus.stopwords.words("english")) #set of english stopwords
    error_wordsu = ['Students', 'Course', '“Recommended', 'Knowledge”'] #words that were incorrectly capitalised

    
    
    string = ''
    
    # Find wordgroups of nouns and conjunctions/articles/prepositions
    for i in range(len(words)):
        if words[i][0] in upper or words[i] in no_cap or words[i] in years or words[i] in years_cond:
            string += words[i] + " "
            #print(string)
        else:
            string = string[:-1]
            string += "=" 
    string = string[:-1] #remove last split sign (=)
    c_names_raw = string.split('=')
    #print(f'c_names_raw: {c_names_raw}')
    
    c_names = []
    
    
    
    # Delete non-capitalised words that embed capitalised words - these are likely not part of the course title (e.g., 'of the', 'Clinical Psychology and the', 'of Evidence-based Clinical Practice: Research Methods/CP')
    for ele in c_names_raw:
        
        words = ele.split()
        #print(f'words:{words}')
        
        while len(words) != 0 and words[0][0] not in upper and words[0] not in years and words[0] not in years_cond: #delete no cap words in the beginning
            del words[0]

        while len(words) != 0 and words[-1][0] not in upper and words[-1] not in years: #delete no cap words in the end and 'first'/'second'/'third' if not used in context of a year
            del words[-1]

        if len(words) == 1 and words[0].lower() in no_cap or len(words) == 1 and words[0].lower() in stop_words or len(words) == 1 and words[0] in error_wordsu: #delete single no cap words and stop words that were capitalised as they opened a sentence XXXtechnically, all verbs and adjectives are probably not course titles
            del words[0]
            
        ele = ' '.join(words)
        
        if len(ele) != 0: #do not append empty course names
            c_names.append(ele)

            
    return c_names

courses = find_c_names(words)
print(courses)

        

['For Brain & Cognition', 'B&C Toolbox: Experimental and Neuroscientific Methods']


In [19]:
# Test: find_credits function

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text
    
    # Entry requirements of the course: full text
    if 'Entry requirements' in info_block: #Find text box with entry requirements
        entry = info_block.replace("Entry requirements", "") #Remove title of the text box
        entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining
        #entry = entry.lower()
        
        #Split into sentences as these contain seperate criteria
        entry = entry.replace(". ", ".").split(".")
        entry = list(filter(None, entry))
        
        
        entry = clean_entry(entry)
        #print(entry)


        
        
        ### HARD REQUIREMENTS
        
        cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
        credits = {}
      
        for sentence in entry:
            words = sentence.split()
            #print(words)

            # COURSE REQUIREMENTS (having completed specific courses)
            courses = find_c_names(words) #gives courses of sentence
            #print(courses)
            
#print(entry) #clean entry
#print(courses) #courses mentioned in the last sentence

['First', 'academic', 'year', 'in', 'Anthropology', '(60', 'EC),', 'Theory', 'and', 'History', 'of', 'Anthropology', '(THA)', 'and/or', 'Historical', 'and', 'Comparative', 'Sociology', '(HCS),', 'a', 'Specialisation', 'Course', 'and', 'Practicing', 'Ethnography']
EC),


In [65]:
# FUNCTION: CREDIT REQUIREMENTS - finished
print(entry)


def find_credits(entry):
    # List of words not capitalised in headings
    conjunctions = ["and", "or", "but", "nor", "yet", "so", "for", "&"]
    articles = ['a', 'an', 'the']
    prepositions = ["in", "to", "of", "at", "by", "up", "for", "off", "on"]
    no_cap = conjunctions + articles + prepositions
    years = ['year', 'first-year', 'second-year', 'third-year'] 
    years_cond = ['first', 'second', 'third', '1st', '2nd', '3rd', 'academic', 'Academic'] #needs to be separate because of problem: Clinical Psychology and the first
    
    cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
    credits = {}
    reverse = False
    for sentence in entry:       
        if 'ec' or 'ects' or 'ect' or 'credits' or 'ect' or 'ecs' in sentence: #check if the entry contains ec information

            words = sentence.split()
            for i in range(len(words)):
                
                if words[i] in cred_l: #find position of 'ec'/...
                    n_ecs = words[i-1] #Find the number of credits (element before 'ec')
                    #print(f'Position of ec {i}')
                    
                    if n_ecs[0] == '(':
                        n_ecs = n_ecs[1:]
                        words = words[::-1]
                        i = len(words)-i
                        reverse = True
                    
                    #print(f'words:{words} and word: {words[i]}')
            
                    co = 1
                    x = 1
                    
                    #Find course or year the credits belong to by going trough the words after ec
                    while x == 1:
                        next_w = words[i+co] #next word


                    # Credits required from first/second/third year
                    
                        first_l = ['first-year', 'first', '1st'] #XXXX might find 'first'/.. that does not belong to year
                        second_l = ['second-year', 'second', '2nd']
                        third_l = ['third-year', 'third', '3rd']
                        
                        if next_w in first_l:
                            credits['First year'] = n_ecs # XXX credits[f'First year {'the_study_programme'}']
                            x = 0
                        elif next_w in second_l:
                            credits['Second year'] = n_ecs 
                            x = 0
                        elif next_w in third_l:
                            credits['Third year'] = n_ecs 
                            x = 0
                            
                            
                    # Credits required from a course(/specialisation): find course(/specialisation) name
                    
                        #check if course name starts by checking capitalisation
                        elif next_w[0] in upper:

                            co2 = 1
                            
                            #Find course names consisting of multiple words and conjunctions, articles and prepositions
                            while words[i+co+co2][0] in upper or words[i+co+co2] in no_cap or words[i+co+co2] in years or words[i+co+co2] in years_cond:
                                next_w += " " + words[i+co+co2]
                                if (i+co+co2+1) == len(words):
                                    #print(words[i+co+co2])
                                    #print('end')
                                    break
                                co2 += 1
                            
                            #Delete conjunctions/articles/prepositions in the end as these are likely not part of 
                            #the course name anymore: e.g 'Clinical Psychology and the'
                            title = next_w.split()
                            if reverse == True: # if reversed because course comes before credits e.g., Research Methods (60), bring back to normal order
                                title = title[::-1]
                            while len(title) != 0 and title[-1] in no_cap or title[-1] in years_cond: 
                                del title[-1]
                            title = ' '.join(title)
                            credits[title] = n_ecs #add title and credits to dictionary
                            x = 0

                        #one word course name
                        else:
                            co+=1
                    words = sentence.split()

    return credits   
                
print(find_credits(entry))


#courses[name]['Entry requirements'] = credits


['60 ec from the first-year Psychology curriculum, including Research Participation', 'Scientific and Statistical Reasoning (7202A701XY), Practical training: Psychological Research (7202A704), as well as ≥ 18 ec of the course package of either Brain & Cognition or Psychological Methods', 'For Brain & Cognition students, the 18 ec must include the course B&C Toolbox: Experimental and Neuroscientific Methods (7202BP04)']
{'First year': '60', 'Brain & Cognition or Psychological Methods': '18', 'B&C Toolbox: Experimental and Neuroscientific Methods': '18'}


In [13]:
# run before testing SCRAPER WORKSPACE: make entry requirements categorical

#indicators = 'priority', 'ec', 'first year', 'research participation', 'only', 'selected into', 
#              'completed', 'second year', course ids, study programme names

url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/100604'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')

#XXX https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595 priority
#XXX https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/100604
#XXX https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/96169
#xxx https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/96129
#XXX https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94883
#XXX https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/101965 
#solved https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/100604 doesnt give credits 


#XXX https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94538 error

In [28]:
# FINAL SCRAPER: ENTRY REQUIREMENTS 
# of one course

upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text
    
    # Entry requirements of the course: full text
    if 'Entry requirements' in info_block: #Find text box with entry requirements
        entry = info_block.replace("Entry requirements", "") #Remove title of the text box
        entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining
        #entry = entry.lower()
        
        #Split into sentences as these contain seperate criteria
        entry = entry.replace(". ", ".").split(".")
        entry = list(filter(None, entry))
        
        
        entry = clean_entry(entry)
        #print(entry)

        ### HARD REQUIREMENTS
        # CREDIT REQUIREMENTS 
        cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
        credits = {}
        credits_dict = find_credits(entry)
        #print(f'Credits for courses: {credits_dict}')
        print(credits_dict)
        

        # COURSE REQUIREMENTS (having completed specific courses); XXX could be moved into function
        #courses = {}
        req_courses = []
        for sentence in entry:
            words = sentence.split()
            new_courses = find_c_names(words)
            #print(f'New courses mentioned: {new_courses}')
            req_courses += new_courses
        #print(f'All courses mentioned: {req_courses}')
        #courses['Courses required'] = req_courses
        print(req_courses)

        
                
        ### SOFT REQUIREMENTS    
            # PRIORISATIONS

                
                
#courses[name]['Entry requirements'] = credits
#courses[name]['Entry requirements'] = 
#courses[name]['Entry requirements'] =

{'first academic year in Anthropology': '60'}
['first academic year in Anthropology', 'Theory and History of Anthropology', 'Historical and Comparative Sociology', 'Specialisation Course and Practicing Ethnography']


In [93]:
#delete and continue on Scraper 2
#FINAL SCRAPER: name, urls, levels and fields (subset of the data) 

potential_levels = ["Bachelor's", "Dual Master's", "Master's", "Exchange Programme Exchange Programme UvA", 
                    "Exchange programme Exchange Programme", "Exchange programme Exchange Programme UvA", 
                    "Exchange Programme Exchange Programme", "pre-Master's programme", "Secondary Subject", 
                    "Bijvak", "Honours MSc", "Research Master's", "Minor"] #tailored to english website
upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"


courses = {}
unique_n = 94500
    
while unique_n < 94550:
    url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/{unique_n}'
    html = simple_get(url)
    dom = BeautifulSoup(html, 'html.parser')
    
    unique_n += 1

    if (dom.find('article', class_ = 'twelve columns')) is None: #no course under this url - go to next url
        print(f'Website doesnt exist: {unique_n}')
        pass

    else: #course exists
        print(f'Website exists: {unique_n}')
        name = dom.find('article', class_ = 'twelve columns').h1.text[1::]
        courses[name] = {}
        #print(name)
        courses[name]['Academic level'] = [] #Add empty list in case there is no data
        courses[name]['Field of study'] = [] #Add empty list in case there is no data
        courses[name]['Entry requirements'] = [] #Add empty list in case there is no data
        courses[name]['url'] = url

        #print(url)
        
        for info_block in dom.find_all('tr', {'class':''}):
            info_block = info_block.text


            #Education level and field of the course
            if 'Is part of' in info_block: #Find text box that contains level and field of the course
                is_part_of = info_block.replace("Is part of", "") #Remove title of the text box

                # Education level of the course e.g., Bachelor's
                levels = [] 
                for l in potential_levels:
                    if l in is_part_of:
                        if 'Exchange' in l: #unified way of describing and exchange programme
                            l = 'Exchange programme'
                        levels.append(l)

                # Field(s) of the course e.g., Business
                for lev in levels: 
                    is_part_of = is_part_of.replace(lev+" ", ";") #replace education level by semicolon to split on the semicolon later
                fields = is_part_of[1::].split(";") #remove first semicolon to not get an empty element in the study fields list 
                                                    #and split on semicolon to get this list
        
                courses[name]['Field of study'] = fields
                courses[name]['Academic level'] = levels
        

            # Entry requirements of the course: full text
            if 'Entry requirements' in info_block: #Find text box with entry requirements
                entry = info_block.replace("Entry requirements", "") #Remove title of the text box
                entry = ' '.join(entry.split()) #remove multiple whitespaces by splitting and joining
                #entry = entry.lower()

                #Split into sentences as these contain seperate criteria
                entry = entry.replace(". ", ".").split(".")
                entry = list(filter(None, entry))


                entry = clean_entry(entry)
                #print(entry)

                ### HARD REQUIREMENTS
                # CREDIT REQUIREMENTS 
                cred_l = ['ec', 'ects', 'ect', 'credits', 'ect', 'ecs']
                credits = {}
                credits_dict = find_credits(entry)
                #print(f'Credits for courses: {credits_dict}')
                #print(credits_dict)


                # COURSE REQUIREMENTS (having completed specific courses); XXX could be moved into function
                req_courses = []
                for sentence in entry:
                    words = sentence.split()
                    new_courses = find_c_names(words)
                    #print(f'New courses mentioned: {new_courses}')
                    req_courses += new_courses
                #print(req_courses)
                entry_req = {}
                entry_req['Required credits'] = credits_dict
                entry_req['Required modules'] = req_courses
                courses[name]['Entry requirements'] = entry_req

print(courses.keys())
print('')
print(courses)
#for name in courses.keys():
#    print(courses[name]['Entry requirements'])
#    print(courses[name]['url'])

Website doesnt exist: 94501
Website exists: 94502
Website doesnt exist: 94503
Website exists: 94504
Website doesnt exist: 94505
Website exists: 94506
Website doesnt exist: 94507
Website doesnt exist: 94508
Website doesnt exist: 94509
Website doesnt exist: 94510
Website doesnt exist: 94511
Website exists: 94512
Website doesnt exist: 94513
Website exists: 94514
Website doesnt exist: 94515
Website doesnt exist: 94516
Website doesnt exist: 94517
Website exists: 94518
Website doesnt exist: 94519
Website exists: 94520
Website doesnt exist: 94521
Website exists: 94522
Website doesnt exist: 94523
Website exists: 94524
Website doesnt exist: 94525
Website exists: 94526
Website exists: 94527
Website doesnt exist: 94528
Website doesnt exist: 94529
Website exists: 94530
Website doesnt exist: 94531
Website exists: 94532
Website doesnt exist: 94533
Website exists: 94534
Website doesnt exist: 94535
Website exists: 94536
Website doesnt exist: 94537
Website exists: 94538


IndexError: list index out of range

In [21]:


















# EXTRAS

url = f'https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595'
html = simple_get(url)
dom = BeautifulSoup(html, 'html.parser')

for info_block in dom.find_all('tr', {'class':''}):
    info_block = info_block.text

    # Coordinator information (optional)
    if 'Lecturer(s)' in info_block:
        #print(info_block) #coord, coord_mail

        
        
#def extract_courses: 



#Special cases

https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/102595 #(MA) or (BA)
https://studiegids.uva.nl/xmlpages/page/2022-2023-en/search-course/course/94517 #Dual Master's / Master's

IndentationError: expected an indented block (2388921358.py, line 22)